# CMS Open Data Tutorial

## First COMETA workshop on artificial intelligence for multi-boson physics
## NIKHEF 2-3 Oct 2024

What we cover here:

* What's available from CMS on the open data portal and how to find it
* The `cernopendata-client` and how to use it
* What's in the CMS NANOAOD format and how to use it
* What's in CMS MINIAOD, how it differs from NANOAOD, and how to enrich NANOAOD
* Then, we'll go to the next notebook and run a ML example using open data

## CMS Open Data and the CERN Open Data Portal

Before we do anything let's go to the [CERN Open Data Portal](https://opendata.cern.ch/) and discover what open data and resources are available.

## Accessing data: the `cernopendata-client`

The [cernopendata-client](https://cernopendata-client.readthedocs.io/en/latest/) is a useful command line tool for accessing data in the CERN Open Data Portal. 


In a terminal in your virtual environment you can install it with the following command:
```
pip install cernopendata-client
```

We'll use it here in the notebook using the command line prompt `!`

Let's run the command with the help option:

In [ ]:
!cernopendata-client --help

Recall that each CMS dataset is associated to a record on the CODP. Let's select a QCD NANOAODSIM dataset:

[Simulated dataset QCD_Pt-15to7000_TuneCP5_Flat2018_13TeV_pythia8 in NANOAODSIM format for 2016 collision data](https://opendata.cern.ch/record/63168)

which is record number 63168.

With the CLI we can list the metadata for this dataset:

In [ ]:
!cernopendata-client get-metadata --recid 63168

Let's list the files available in this record:

In [ ]:
!cernopendata-client get-file-locations --recid 63168 --verbose

By default the files are given with the HTTP protocol. If you wish to use XRootD: 

In [ ]:
!cernopendata-client get-file-locations --recid 63168 --protocol xrootd --verbose

For reference, here's a a bit of code to fetch the available file from a record and output it to a list:

In [ ]:
import subprocess

recid = 63168

command = ['cernopendata-client', 'get-file-locations', '--recid', '63168', '--protocol', 'xrootd']
result = subprocess.run(command, capture_output=True, text=True)
root_files = result.stdout.splitlines()
                    
print(root_files)

Now let's download a file, one of the smaller ones. Note that the `cernopendata-client` has a download files option (see below) but since we know which file we want well use `curl` instead. 

In [ ]:
!cernopendata-client download-files --help

In [ ]:
!curl -O http://opendata.cern.ch/eos/opendata/cms/mc/RunIISummer20UL16NanoAODv9/QCD_Pt-15to7000_TuneCP5_Flat2018_13TeV_pythia8/NANOAODSIM/106X_mcRun2_asymptotic_v17-v1/270000/397D1673-167A-CF46-9E79-D7069D9AC359.root

## Exploring the data: the CMS NANOAOD format

The NANOAOD files can of course be opened and explored using ROOT, in particular [PyROOT](https://root.cern/manual/python/). Here we'll instead use the SciKit-HEP tools [uproot](https://github.com/scikit-hep/uproot5) ("ROOT I/O in pure Python and NumPy") and [awkward](https://github.com/scikit-hep/awkward) ("Manipulate JSON-like data with NumPy-like idioms").

In [ ]:
import numpy as np
import matplotlib.pylab as plt

import uproot
import awkward as ak

Let's first open the file we've downloaded and see what's in it:

In [ ]:
infile_name = '397D1673-167A-CF46-9E79-D7069D9AC359.root'
infile = uproot.open(infile_name)

keys = infile.keys()
print(keys)

If you were using ROOT and opened up the file with a `TBrowser` you would see something like this:

![nanoaod in the tbrowser](imgs/nano-root.png)

Since we're using `uproot/awkward` let's see what we have:

In [ ]:
events = infile['Events']
events.show()

Let's examine the key-value combinations:

In [ ]:
events.keys()

In [ ]:
events.values()

What does all this mean? Recall that thankfully in the [dataset record](https://opendata.cern.ch/record/63168) there is a list of file variables (with explanations) under `Dataset semantics`. The link is [here](https://opendata.cern.ch/eos/opendata/cms/dataset-semantics/NanoAODSIM/63168/QCD_Pt-15to7000_TuneCP5_Flat2018_13TeV_pythia8_doc.html).

![nanoaod dataset semantics](imgs/nano-jet.png)

Let's extract and plot some variables:

* jet multiplicity

* jet pt

* jet eta

We'll read them out as `awkward` arrays:

In [ ]:
jet_mult = events['Jet_nConstituents'].array()
jet_pt = events['Jet_pt'].array()
jet_eta = events['Jet_eta'].array()

Note the structure of the array, in which each element of array corresponds to an event and the size of each element
depends on the number of jets in the event:

In [ ]:
jet_mult

Note that we have to use `ak.flatten` to flatten the array structure before we create a histogram:

In [ ]:
ak.flatten(jet_mult)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(20,10))

axes[0].hist(ak.flatten(jet_mult), bins=120, range=(0,120))
axes[0].set_xlabel('jet_mult')

axes[1].hist(ak.flatten(jet_pt), bins=80, range=(0,8000))
axes[1].set_xlabel('jet_pt')
axes[1].set_yscale('log')

axes[2].hist(ak.flatten(jet_eta), bins=100, range=(-6,6))
axes[2].set_xlabel('jet_eta')

It will be convenient to only analyze jet-related variables so let's make a crude selection of `Jet_` keys and collect the objects into a collection of arrays:

In [ ]:
jet_keys = [jk for jk in events.keys() if 'Jet_' in jk]
arrays = events.arrays(jet_keys, library="ak")

In [ ]:
arrays

For reference, you can also easily convert your collection of arrays into a Pandas DataFrame:

In [ ]:
df = events.arrays(jet_keys, library="pd")

In [ ]:
df

We'll stick with `awkward` and add selections on G and UDS jets to the arrays so that we can distinguish between them later:

In [ ]:
arrays['Jet_isG'] = ak.where(arrays['Jet_partonFlavour'] == 21, True, False)

arrays['Jet_isUDS'] = ak.where(
    (np.abs(arrays['Jet_partonFlavour']) == 1) | (np.abs(arrays['Jet_partonFlavour']) == 2) | (np.abs(arrays['Jet_partonFlavour']) == 3),
    True, False
)

One of the variables for in the `Jet` object is `Jet_qgl` which is the result of a relatively simple quark/gluon jet classifier which is based on three jet variables (which we describe in more detail in the next notebook). This discriminator assigns a probability of a jet coming from a light quark. Let's plot what it looks like for this NANOAOD sample:

In [ ]:
plt.clf()
binning = np.arange(0.0, 1.0, 0.04)

plt.hist(
    ak.flatten(arrays['Jet_qgl'][ak.where(arrays['Jet_isG'], True, False)]), 
    bins=binning, alpha=0.8, label='Gluon', density=1 )

plt.hist(
    ak.flatten(arrays['Jet_qgl'][ak.where(arrays['Jet_isUDS'], True, False)]), 
    bins=binning, alpha=0.8, label='Quark', density=1 )

plt.legend()

plt.xlabel('Discriminator output')
plt.title('Quark/gluon likelihood');

We'll return to jet discriminators in the next notebook but before then let's quickly
demonstrate how to make some simple selections:

* nJets >= 3
* jet_pt > 30 GeV
* abs(jet_eta) < 2.5

In [ ]:
njet_cut = ak.where(
    ak.num(arrays['Jet_pt']) >= 3,
    True, False
) 

print(njet_cut)

In [ ]:
arrays = arrays[njet_cut]

pt_cut = ak.all(arrays['Jet_pt'] > 30, axis=1)
eta_cut = ak.all(np.abs(arrays['Jet_eta']) < 2.5, axis=1)

arrays = arrays[pt_cut & eta_cut]

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20,10))

axes[0].hist(ak.flatten(arrays['Jet_pt']), bins=80, range=(0,8000))
axes[0].set_xlabel('jet_pt')
axes[0].set_yscale('log')

axes[1].hist(ak.flatten(arrays['Jet_eta']), bins=100, range=(-6,6))
axes[1].set_xlabel('jet_eta')

## CMS MINIAOD vs NANOAOD

Hopefully we've shown some of nice features of NANOAOD including:
* it's relatively lightweight
* it has a relatively simple structure
* it doesn't require complex CMS software for analysis

However, you may find that the NANOAOD doesn't have some information that you need for your study. Content-wise, one major difference between NANO and MINIAOD is that MINIAOD contains most of the constituents of a physics object (such as tracks and/or calorimeter clusters) whereas NANOAOD only contains some information about them. 
If you need this information, you may need either to process MINIAOD directly.

Objects in MINIAOD are created as C++ classes in the CMS software framework CMSSW. Analysing MINIAOD requires CMSSW software. CMS provides Docker container images with the appropriate CMS software environment as well as example code.

From the [NANOAOD dataset record](https://opendata.cern.ch/record/63168) we can find the [MINIAOD parent dataset](https://opendata.cern.ch/record/63167) from which the NANOAOD was derived.

From this record we see which Docker container we can use to analyze this data:

![docker miniaod](imgs/docker-miniaod.png)

Analysis of MINIAOD is beyond the scope of this notebook-based tutorial but would be worth checking out [this guide](https://opendata.cern.ch/docs/cms-getting-started-miniaod) to getting started with MINIAOD.

If one does open one a MINIAOD root file in the CMSSW Docker container one can inspect the structure of the format like so:

```
(/code/CMSSW_10_6_30/src) edmDumpEventContent root://eospublic.cern.ch//eos/opendata/cms/mc/RunIISummer20UL16MiniAODv2/QCD_Pt-15to7000_TuneCP5_Flat2018_13TeV_pythia8/MINIAODSIM/106X_mcRun2_asymptotic_v17-v1/130000/1092DDD2-A0E6-DE47-8E72-807987BC5F99.root
Type                                  Module                      Label             Process
----------------------------------------------------------------------------------------------
edm::TriggerResults                   "TriggerResults"            ""                "HLT"
BXVector<GlobalAlgBlk>                "gtStage2Digis"             ""                "RECO"
BXVector<GlobalExtBlk>                "gtStage2Digis"             ""                "RECO"
BXVector<l1t::EGamma>                 "caloStage2Digis"           "EGamma"          "RECO"
BXVector<l1t::EtSum>                  "caloStage2Digis"           "EtSum"           "RECO"
BXVector<l1t::Jet>                    "caloStage2Digis"           "Jet"             "RECO"
BXVector<l1t::Muon>                   "gmtStage2Digis"            "Muon"            "RECO"
BXVector<l1t::Tau>                    "caloStage2Digis"           "Tau"             "RECO"
HcalNoiseSummary                      "hcalnoise"                 ""                "RECO"
L1GlobalTriggerReadoutRecord          "gtDigis"                   ""                "RECO"
double                                "fixedGridRhoAll"           ""                "RECO"
double                                "fixedGridRhoFastjetAll"    ""                "RECO"
double                                "fixedGridRhoFastjetAllCalo"   ""                "RECO"
double                                "fixedGridRhoFastjetAllTmp"   ""                "RECO"
double                                "fixedGridRhoFastjetCentral"   ""                "RECO"
double                                "fixedGridRhoFastjetCentralCalo"   ""                "RECO"
double                                "fixedGridRhoFastjetCentralChargedPileUp"   ""                "RECO"
double                                "fixedGridRhoFastjetCentralNeutral"   ""                "RECO"
edm::TriggerResults                   "TriggerResults"            ""                "RECO"
reco::BeamHaloSummary                 "BeamHaloSummary"           ""                "RECO"
reco::BeamSpot                        "offlineBeamSpot"           ""                "RECO"
reco::CSCHaloData                     "CSCHaloData"               ""                "RECO"
vector<CTPPSLocalTrackLite>           "ctppsLocalTrackLiteProducer"   ""                "RECO"
vector<LumiScalers>                   "scalersRawToDigi"          ""                "RECO"
vector<l1extra::L1EmParticle>         "l1extraParticles"          "Isolated"        "RECO"
vector<l1extra::L1EmParticle>         "l1extraParticles"          "NonIsolated"     "RECO"
vector<l1extra::L1EtMissParticle>     "l1extraParticles"          "MET"             "RECO"
vector<l1extra::L1EtMissParticle>     "l1extraParticles"          "MHT"             "RECO"
vector<l1extra::L1HFRings>            "l1extraParticles"          ""                "RECO"
vector<l1extra::L1JetParticle>        "l1extraParticles"          "Central"         "RECO"
vector<l1extra::L1JetParticle>        "l1extraParticles"          "Forward"         "RECO"
vector<l1extra::L1JetParticle>        "l1extraParticles"          "IsoTau"          "RECO"
vector<l1extra::L1JetParticle>        "l1extraParticles"          "Tau"             "RECO"
vector<l1extra::L1MuonParticle>       "l1extraParticles"          ""                "RECO"
vector<reco::Conversion>              "gsfTracksOpenConversions"   "gsfTracksOpenConversions"   "RECO"
vector<reco::ForwardProton>           "ctppsProtons"              "multiRP"         "RECO"
vector<reco::ForwardProton>           "ctppsProtons"              "singleRP"        "RECO"
vector<reco::Track>                   "displacedStandAloneMuons"   ""                "RECO"
BXVector<GlobalExtBlk>                "simGtExtUnprefireable"     ""                "PAT"
double                                "prefiringweight"           "nonPrefiringProb"   "PAT"
double                                "prefiringweight"           "nonPrefiringProbDown"   "PAT"
double                                "prefiringweight"           "nonPrefiringProbUp"   "PAT"
edm::Association<vector<reco::DeDxHitInfo> >    "isolatedTracks"            ""                "PAT"
edm::OwnVector<TrackingRecHit,edm::ClonePolicy<TrackingRecHit> >    "slimmedMuonTrackExtras"    ""                "PAT"
edm::OwnVector<reco::BaseTagInfo,edm::ClonePolicy<reco::BaseTagInfo> >    "slimmedJetsPuppi"          "tagInfos"        "PAT"
edm::RangeMap<CSCDetId,edm::OwnVector<CSCSegment,edm::ClonePolicy<CSCSegment> >,edm::ClonePolicy<CSCSegment> >    "slimmedMuons"              ""                "PAT"
edm::RangeMap<DTChamberId,edm::OwnVector<DTRecSegment4D,edm::ClonePolicy<DTRecSegment4D> >,edm::ClonePolicy<DTRecSegment4D> >    "slimmedMuons"              ""
"PAT"
edm::SortedCollection<EcalRecHit,edm::StrictWeakOrdering<EcalRecHit> >    "reducedEgamma"             "reducedEBRecHits"   "PAT"
edm::SortedCollection<EcalRecHit,edm::StrictWeakOrdering<EcalRecHit> >    "reducedEgamma"             "reducedEERecHits"   "PAT"
edm::SortedCollection<EcalRecHit,edm::StrictWeakOrdering<EcalRecHit> >    "reducedEgamma"             "reducedESRecHits"   "PAT"
edm::SortedCollection<HBHERecHit,edm::StrictWeakOrdering<HBHERecHit> >    "reducedEgamma"             "reducedHBHEHits"   "PAT"
edm::SortedCollection<HBHERecHit,edm::StrictWeakOrdering<HBHERecHit> >    "slimmedHcalRecHits"        "reducedHcalRecHits"   "PAT"
edm::SortedCollection<HFRecHit,edm::StrictWeakOrdering<HFRecHit> >    "slimmedHcalRecHits"        "reducedHcalRecHits"   "PAT"
edm::SortedCollection<HORecHit,edm::StrictWeakOrdering<HORecHit> >    "slimmedHcalRecHits"        "reducedHcalRecHits"   "PAT"
edm::TriggerResults                   "TriggerResults"            ""                "PAT"
edm::ValueMap<float>                  "offlineSlimmedPrimaryVertices"   ""                "PAT"
pat::PackedTriggerPrescales           "patTrigger"                ""                "PAT"
pat::PackedTriggerPrescales           "patTrigger"                "l1max"           "PAT"
pat::PackedTriggerPrescales           "patTrigger"                "l1min"           "PAT"
vector<CTPPSLocalTrackLite>           "ctppsLocalTrackLiteProducer"   ""                "PAT"
vector<pat::CompositeCandidate>       "oniaPhotonCandidates"      "conversions"     "PAT"
vector<pat::Electron>                 "slimmedElectrons"          ""                "PAT"
vector<pat::Electron>                 "slimmedLowPtElectrons"     ""                "PAT"
vector<pat::IsolatedTrack>            "isolatedTracks"            ""                "PAT"
vector<pat::Jet>                      "slimmedJets"               ""                "PAT"
vector<pat::Jet>                      "slimmedJetsAK8"            ""                "PAT"
vector<pat::Jet>                      "slimmedJetsPuppi"          ""                "PAT"
vector<pat::Jet>                      "slimmedJetsAK8PFPuppiSoftDropPacked"   "SubJets"         "PAT"
vector<pat::MET>                      "slimmedMETs"               ""                "PAT"
vector<pat::MET>                      "slimmedMETsNoHF"           ""                "PAT"
vector<pat::MET>                      "slimmedMETsPuppi"          ""                "PAT"
vector<pat::Muon>                     "slimmedMuons"              ""                "PAT"
vector<pat::PackedCandidate>          "lostTracks"                ""                "PAT"
vector<pat::PackedCandidate>          "packedPFCandidates"        ""                "PAT"
vector<pat::PackedCandidate>          "lostTracks"                "eleTracks"       "PAT"
vector<pat::Photon>                   "slimmedOOTPhotons"         ""                "PAT"
vector<pat::Photon>                   "slimmedPhotons"            ""                "PAT"
vector<pat::Tau>                      "slimmedTaus"               ""                "PAT"
vector<pat::Tau>                      "slimmedTausBoosted"        ""                "PAT"
vector<pat::TriggerObjectStandAlone>    "slimmedPatTrigger"         ""                "PAT"
vector<reco::CaloCluster>             "reducedEgamma"             "reducedEBEEClusters"   "PAT"
vector<reco::CaloCluster>             "reducedEgamma"             "reducedESClusters"   "PAT"
vector<reco::CaloCluster>             "reducedEgamma"             "reducedOOTEBEEClusters"   "PAT"
vector<reco::CaloCluster>             "reducedEgamma"             "reducedOOTESClusters"   "PAT"
vector<reco::CaloJet>                 "slimmedCaloJets"           ""                "PAT"
vector<reco::Conversion>              "reducedEgamma"             "reducedConversions"   "PAT"
vector<reco::Conversion>              "reducedEgamma"             "reducedSingleLegConversions"   "PAT"
vector<reco::DeDxHitInfo>             "isolatedTracks"            ""                "PAT"
vector<reco::ForwardProton>           "ctppsProtons"              "multiRP"         "PAT"
vector<reco::ForwardProton>           "ctppsProtons"              "singleRP"        "PAT"
vector<reco::GsfElectronCore>         "reducedEgamma"             "reducedGedGsfElectronCores"   "PAT"
vector<reco::GsfTrack>                "reducedEgamma"             "reducedGsfTracks"   "PAT"
vector<reco::PhotonCore>              "reducedEgamma"             "reducedGedPhotonCores"   "PAT"
vector<reco::PhotonCore>              "reducedEgamma"             "reducedOOTPhotonCores"   "PAT"
vector<reco::SuperCluster>            "reducedEgamma"             "reducedOOTSuperClusters"   "PAT"
vector<reco::SuperCluster>            "reducedEgamma"             "reducedSuperClusters"   "PAT"
vector<reco::TrackExtra>              "slimmedMuonTrackExtras"    ""                "PAT"
vector<reco::Vertex>                  "offlineSlimmedPrimaryVertices"   ""                "PAT"
vector<reco::VertexCompositePtrCandidate>    "slimmedKshortVertices"     ""                "PAT"
vector<reco::VertexCompositePtrCandidate>    "slimmedLambdaVertices"     ""                "PAT"
vector<reco::VertexCompositePtrCandidate>    "slimmedSecondaryVertices"   ""                "PAT"
vector<string>                        "slimmedPatTrigger"         "filterLabels"    "PAT"
unsigned int                          "bunchSpacingProducer"      ""                "PAT"
```

It is worth mentioning that some NANOAOD samples have been produced from MINAOD that have been enriched with Particle Flow (what's that? See [here](https://cms-opendata-workshop.github.io/workshop2024-lesson-physics-objects/instructor/01-introduction.html) to learn more) information. It would be a useful exercise for the reader to try the workflow described [here](https://opendata.cern.ch/record/12504) to produce your own enhanced NANOAOD and to learn how to make your own customizations.